<a href="https://colab.research.google.com/github/doomba32/python/blob/main/%EA%B2%BD%EA%B8%B0%EB%8F%84%EB%A7%9B%EC%A7%91%ED%98%84%ED%99%A9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#pandas 모듈을 pd 라는 이름으로 import
import pandas as pd

In [4]:
#Restaurant.csv 파일로부터 읽은 data를 restaurant 변수에 저장
restaurant = pd.read_csv('http://doomba32.github.io/python/data/Restaurant.csv', encoding='euc-kr')

#restaurant의 앞쪽 데이터 확인
restaurant.head()

,시군명,음식점명,맛집전화번호,대표음식명,소재지우편번호,소재지도로명주소,소재지지번주소,WGS84위도,WGS84경도
0,가평군,가평축협 한우명가,031-581-1592,푸른연잎한우명품꽃등심,12422,경기도 가평군 가평읍 달전로 19,경기도 가평군 가평읍 달전리 382-1번지,37.815844,127.516128
1,고양시,청정바지락칼국수,031-912-7676,천년초들깨수제비,10359,경기도 고양시 일산동구 일산로463번길 7,경기도 고양시 일산동구 정발산동 1148번지,37.673707,126.775375
2,고양시,양촌리아구,031-911-0430,아구탕,10218,경기도 고양시 일산서구 대화2로 152,경기도 고양시 일산서구 대화동 762-3번지,37.671931,126.736219
3,고양시,정통중화요리 남궁,031-911-3702,"해물고추짬뽕, 양장피잡채",10367,경기도 고양시 일산서구 일산로 682,경기도 고양시 일산서구 대화동 2101번지,37.682042,126.753550
4,고양시,야구장농원,031-964-2884,오리진흙구이,10313,경기도 고양시 일산동구 견달산로 351,경기도 고양시 일산동구 문봉동 102-1번지,37.697102,126.819819
